In [1]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine
import requests
import json
import pymysql
import numpy as np
import pickle
import os

/tmp/ipykernel_196375/407637579.py:2 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

In [8]:
# current_dir = os.path.dirname(os.path.abspath(__file__))
# info_path = os.path.join(current_dir, 'DB_INFO.pkl')

with open('./DB_INFO.pkl', "rb") as file:
    USER = pickle.load(file)
    PASSWD = pickle.load(file)
    HOST = pickle.load(file)
    PORT = pickle.load(file)
    NAME = pickle.load(file)

In [3]:
result = pd.read_csv('./travel_spot_v1.csv', index_col=0)
len(result)

21034

In [5]:
result['id'] = result['id'].apply(lambda x: x*10) # 전체의 id는 기본값 *10으로 구분

In [6]:
result.head(1)

,id,city,cityCode,contentType,title,catchtitle,overView,treatMenu,conLike,conRead,...,imgPath,addr,info,parking,useTime,tagName,detail,lat,lon,tf
0,28379640,전체,1,39,봉희설렁탕 본점,40년 전통의 은평구 설렁탕집,봉희설렁탕은 서울 은평구 신사동에 있다. 6호선 새절역 4번 출구 인근에 있다. 이...,"설렁탕, 양곰탕, 설곰탕, 얼큰설곰탕, 특설렁탕, 꼬리곰탕, 양무침, 황태구이, 도...",1,351,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 은평구 증산로 413 동현빌딩,02-302-9754,있음 (발렛파킹),06:00~22:00,"백련산맛집,봉희설렁탕,불광천가로수길맛집,새절역맛집,설렁탕맛집,설렁탕맛집,음식",부대시설:무선인터넷,37.593186,126.913862,"백련산맛집,봉희설렁탕,불광천가로수길맛집,새절역맛집,설렁탕맛집,설렁탕맛집,음식설렁탕,..."


In [19]:
df = result
df2 = result.copy()
df2['city'] = '전체'
df2['id'] = df2['id'].apply(lambda x: x*10) # 전체의 id는 기본값 *10으로 구분
joined_df = pd.concat([df,df2], axis=0, ignore_index=True)

In [21]:
joined_df

,id,city,cityCode,contentType,title,catchtitle,overView,treatMenu,conLike,conRead,...,imgPath,addr,info,parking,useTime,tagName,detail,lat,lon,tf
0,2837964,전체,1,39,봉희설렁탕 본점,40년 전통의 은평구 설렁탕집,봉희설렁탕은 서울 은평구 신사동에 있다. 6호선 새절역 4번 출구 인근에 있다. 이...,"설렁탕, 양곰탕, 설곰탕, 얼큰설곰탕, 특설렁탕, 꼬리곰탕, 양무침, 황태구이, 도...",1,351,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 은평구 증산로 413 동현빌딩,02-302-9754,있음 (발렛파킹),06:00~22:00,"백련산맛집,봉희설렁탕,불광천가로수길맛집,새절역맛집,설렁탕맛집,설렁탕맛집,음식",부대시설:무선인터넷,37.593186,126.913862,"백련산맛집,봉희설렁탕,불광천가로수길맛집,새절역맛집,설렁탕맛집,설렁탕맛집,음식설렁탕,..."
1,2845652,전체,1,39,벙구갈비 응암점,응암동 전통 수제 숙성 갈비 전문점,벙구갈비 응암점은 지하철 6호선 응암역 4번 출구 은명초등학교 부근에 있다. 매장 ...,"수제갈비 (350g), 수제생갈비 (300g), 수제그린포크 (250g), \n수제...",1,492,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 은평구 응암로 269,02-352-5092,있음,매일 11:00 ~ 23:00,"벙구갈비,벙구갈비응암점,은평구맛집,은평구맛집,음식,응암동맛집,응암역맛집",화장실:남녀 구분,37.596061,126.918784,"벙구갈비,벙구갈비응암점,은평구맛집,은평구맛집,음식,응암동맛집,응암역맛집수제갈비 (3..."
2,2757826,전체,1,39,영동설렁탕,NaN,30여 년간 한자리에서 운영된 이 식당은 아직도 허름한 식당 외관을 고수하고 있다....,국밥,0,959,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 서초구 강남대로101안길 24,02-543-4716,있음(40대주차가능),00:00 - 24:00,"서초국밥,서초설렁탕,서초수육,서초영동설렁탕,영동설렁탕,음식",화장실:있음(남/녀 구분)\n,37.516211,127.017402,"서초국밥,서초설렁탕,서초수육,서초영동설렁탕,영동설렁탕,음식국밥"
3,398352,전체,1,39,마복림원조할머니집떡볶이,NaN,"50년 이어온 전통의 맛, 떡볶이 대표 브랜드로 유명하다. 서울특별시 중구에 있는 ...",떡볶이 / 튀김,2,2238,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 중구 다산로35길 5,02-2232-8930,가능,09:00-00:00,"떡볶이,마복림떡볶이,맛집,서울맛집추천,신당동맛집,음식","화장실:업소내, 남녀공용 화장실",37.563222,127.015115,"떡볶이,마복림떡볶이,맛집,서울맛집추천,신당동맛집,음식떡볶이 / 튀김"
4,2721659,전체,1,39,태극당,서울에서 가장 맛있고 오래된 빵집,태극당은 서울에서 가장 오래된 빵집으로 1945년 광복의 때를 기다린 듯 우리 민족...,모나카 / 버터케익 / 야채 사라다 / 고방 카스테라 등,7,1296,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 중구 동호로24길 7,02-2279-3152,NaN,08:00~21:00,"데이트,베이커리,복고풍,신선한,옛날맛,옛날빵,옛날빵집,옛스러운분위기,음식,체인점",화장실:남/녀 화장실 구분,37.559581,127.005108,"데이트,베이커리,복고풍,신선한,옛날맛,옛날빵,옛날빵집,옛스러운분위기,음식,체인점모나..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42063,1264400,전체,39,12,안덕계곡,자연 풍광을 즐기는 계곡,안덕계곡은 서귀포 안덕면의 감산리 마을을 지나 바다로 유입되는 창고천 하류에 형성된...,NaN,2,1974,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 서귀포시 안덕면 일주서로 1524,064-710-6653 (제주도청세계유산본부:상록수림담당)<br>\n064-732-...,주차 가능,상시개방,"경치좋은곳,계곡,관광지,생태관광지,안덕계곡,연인과함께,자연,자연좋은곳,제주권",입 장 료:무료,33.258440,126.352993,NaN
42064,7595950,전체,39,14,제주도립미술관,제주다운 미술관,제주도립미술관은 아름답고 청정한 제주의 자연 속에 세워진 ‘예술의 빛과 숨결이 넘치...,NaN,2,1662,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 제주시 1100로 2894-78,064-710-4300,가능,09:00~18:00 (7월~9월 09:00~20:00)<br>\n※ 매표는 관람시...,"문화시설,문화예술,문화예술공간,미술관,복합문화공간,산책하기좋은곳,실내여행지,연인과함...","대관안내:[대관시설] 기획전시실(대), 기획전시실(소), 시민갤러리, 강당, 세미나...",33.452549,126.489752,NaN
42065,1264430,전체,39,12,화순금모래해변,검은 모래밭이 펼쳐진 해변이 아름다운 곳,"화순금모래 해변은 99,174m²(약 3만 평)의 검은 모래밭이 펼쳐진 곳으로 주변...",NaN,3,2102,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 서귀포시 안덕면 화순해안로,064-760-4372 (안덕면 사무소),있음,상시개방(해수욕장은 매년 상이),"가족여행,관광지,바다풍경,바다풍경여행,아이와함께,여름방학가볼만한곳,여름여행,여름휴가...",화장실:있음(남/녀구분),33.240700,126.333515,NaN
42066,24148120,전체,39,12,신산공원,여유롭게 산책하며 휴식할 수 있는 공원,신산공원은 올림픽 성화의 국내도착을 기념하는 기념물이 세워져 있는 88 서울 올림픽...,NaN,1,1125,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 제주시 신산로 92-12,064-726-0885,있음,NaN,"걷기길,관광지,나들이,당일치기여행,산책길,산책코스,산책하기,산책하기좋은곳,신산공원,...",화장실:있음(남/녀구분),33.505280,126.533792,NaN


In [13]:
df['id'] = df['id'].apply(lambda x: x*10)

In [15]:
df

,id,city,cityCode,contentType,title,catchtitle,overView,treatMenu,conLike,conRead,...,imgPath,addr,info,parking,useTime,tagName,detail,lat,lon,tf
0,28379640,전체,1,39,봉희설렁탕 본점,40년 전통의 은평구 설렁탕집,봉희설렁탕은 서울 은평구 신사동에 있다. 6호선 새절역 4번 출구 인근에 있다. 이...,"설렁탕, 양곰탕, 설곰탕, 얼큰설곰탕, 특설렁탕, 꼬리곰탕, 양무침, 황태구이, 도...",1,351,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 은평구 증산로 413 동현빌딩,02-302-9754,있음 (발렛파킹),06:00~22:00,"백련산맛집,봉희설렁탕,불광천가로수길맛집,새절역맛집,설렁탕맛집,설렁탕맛집,음식",부대시설:무선인터넷,37.593186,126.913862,"백련산맛집,봉희설렁탕,불광천가로수길맛집,새절역맛집,설렁탕맛집,설렁탕맛집,음식설렁탕,..."
1,28456520,전체,1,39,벙구갈비 응암점,응암동 전통 수제 숙성 갈비 전문점,벙구갈비 응암점은 지하철 6호선 응암역 4번 출구 은명초등학교 부근에 있다. 매장 ...,"수제갈비 (350g), 수제생갈비 (300g), 수제그린포크 (250g), \n수제...",1,492,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 은평구 응암로 269,02-352-5092,있음,매일 11:00 ~ 23:00,"벙구갈비,벙구갈비응암점,은평구맛집,은평구맛집,음식,응암동맛집,응암역맛집",화장실:남녀 구분,37.596061,126.918784,"벙구갈비,벙구갈비응암점,은평구맛집,은평구맛집,음식,응암동맛집,응암역맛집수제갈비 (3..."
2,27578260,전체,1,39,영동설렁탕,NaN,30여 년간 한자리에서 운영된 이 식당은 아직도 허름한 식당 외관을 고수하고 있다....,국밥,0,959,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 서초구 강남대로101안길 24,02-543-4716,있음(40대주차가능),00:00 - 24:00,"서초국밥,서초설렁탕,서초수육,서초영동설렁탕,영동설렁탕,음식",화장실:있음(남/녀 구분)\n,37.516211,127.017402,"서초국밥,서초설렁탕,서초수육,서초영동설렁탕,영동설렁탕,음식국밥"
3,3983520,전체,1,39,마복림원조할머니집떡볶이,NaN,"50년 이어온 전통의 맛, 떡볶이 대표 브랜드로 유명하다. 서울특별시 중구에 있는 ...",떡볶이 / 튀김,2,2238,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 중구 다산로35길 5,02-2232-8930,가능,09:00-00:00,"떡볶이,마복림떡볶이,맛집,서울맛집추천,신당동맛집,음식","화장실:업소내, 남녀공용 화장실",37.563222,127.015115,"떡볶이,마복림떡볶이,맛집,서울맛집추천,신당동맛집,음식떡볶이 / 튀김"
4,27216590,전체,1,39,태극당,서울에서 가장 맛있고 오래된 빵집,태극당은 서울에서 가장 오래된 빵집으로 1945년 광복의 때를 기다린 듯 우리 민족...,모나카 / 버터케익 / 야채 사라다 / 고방 카스테라 등,7,1296,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,서울특별시 중구 동호로24길 7,02-2279-3152,NaN,08:00~21:00,"데이트,베이커리,복고풍,신선한,옛날맛,옛날빵,옛날빵집,옛스러운분위기,음식,체인점",화장실:남/녀 화장실 구분,37.559581,127.005108,"데이트,베이커리,복고풍,신선한,옛날맛,옛날빵,옛날빵집,옛스러운분위기,음식,체인점모나..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21029,1264400,제주,39,12,안덕계곡,자연 풍광을 즐기는 계곡,안덕계곡은 서귀포 안덕면의 감산리 마을을 지나 바다로 유입되는 창고천 하류에 형성된...,NaN,2,1974,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 서귀포시 안덕면 일주서로 1524,064-710-6653 (제주도청세계유산본부:상록수림담당)<br>\n064-732-...,주차 가능,상시개방,"경치좋은곳,계곡,관광지,생태관광지,안덕계곡,연인과함께,자연,자연좋은곳,제주권",입 장 료:무료,33.258440,126.352993,NaN
21030,7595950,제주,39,14,제주도립미술관,제주다운 미술관,제주도립미술관은 아름답고 청정한 제주의 자연 속에 세워진 ‘예술의 빛과 숨결이 넘치...,NaN,2,1662,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 제주시 1100로 2894-78,064-710-4300,가능,09:00~18:00 (7월~9월 09:00~20:00)<br>\n※ 매표는 관람시...,"문화시설,문화예술,문화예술공간,미술관,복합문화공간,산책하기좋은곳,실내여행지,연인과함...","대관안내:[대관시설] 기획전시실(대), 기획전시실(소), 시민갤러리, 강당, 세미나...",33.452549,126.489752,NaN
21031,1264430,제주,39,12,화순금모래해변,검은 모래밭이 펼쳐진 해변이 아름다운 곳,"화순금모래 해변은 99,174m²(약 3만 평)의 검은 모래밭이 펼쳐진 곳으로 주변...",NaN,3,2102,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 서귀포시 안덕면 화순해안로,064-760-4372 (안덕면 사무소),있음,상시개방(해수욕장은 매년 상이),"가족여행,관광지,바다풍경,바다풍경여행,아이와함께,여름방학가볼만한곳,여름여행,여름휴가...",화장실:있음(남/녀구분),33.240700,126.333515,NaN
21032,24148120,제주,39,12,신산공원,여유롭게 산책하며 휴식할 수 있는 공원,신산공원은 올림픽 성화의 국내도착을 기념하는 기념물이 세워져 있는 88 서울 올림픽...,NaN,1,1125,...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,제주특별자치도 제주시 신산로 92-12,064-726-0885,있음,NaN,"걷기길,관광지,나들이,당일치기여행,산책길,산책코스,산책하기,산책하기좋은곳,신산공원,...",화장실:있음(남/녀구분),33.505280,126.533792,NaN


In [16]:
len(df)

21034

In [ ]:
    port = PORT
    host = HOST
    user = USER
    password = PASSWD
    database = NAME
    charset = 'utf8mb4'
    
    conn = pymysql.connect(host=host,user=user,password=password,database=database,charset=charset)


    joined_df = joined_df.fillna('')
    try:
        with conn.cursor() as cursor:
            for inx, row in joined_df.iterrows():
                # 모든 레코드를 업데이트하고, 없는 경우에는 새로 삽입
                query = f"""
                    INSERT INTO TRAVEL_SPOT (
                        SPOT_ID, SPOT_CITY, SPOT_CITY_CODE, SPOT_CITY_CONTENT_TYPE,
                        SPOT_TITLE, SPOT_CATCHTITLE, SPOT_OVERVIEW, SPOT_TREATMENU,
                        SPOT_CONLIKE, SPOT_CONREAD, SPOT_CONSHARE, SPOT_IMGPATH,
                        SPOT_ADDR, SPOT_INFOCENTER, SPOT_PARKING, SPOT_USETIME,
                        SPOT_TAGNAME, SPOT_DETAIL, SPOT_LON, SPOT_LAT
                    )
                    VALUES (
                        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                    )
                    ON DUPLICATE KEY UPDATE
                        SPOT_CITY = VALUES(SPOT_CITY),
                        SPOT_CITY_CODE = VALUES(SPOT_CITY_CODE),
                        SPOT_CITY_CONTENT_TYPE = VALUES(SPOT_CITY_CONTENT_TYPE),
                        SPOT_TITLE = VALUES(SPOT_TITLE),
                        SPOT_CATCHTITLE = VALUES(SPOT_CATCHTITLE),
                        SPOT_OVERVIEW = VALUES(SPOT_OVERVIEW),
                        SPOT_TREATMENU = VALUES(SPOT_TREATMENU),
                        SPOT_CONLIKE = VALUES(SPOT_CONLIKE),
                        SPOT_CONREAD = VALUES(SPOT_CONREAD),
                        SPOT_CONSHARE = VALUES(SPOT_CONSHARE),
                        SPOT_IMGPATH = VALUES(SPOT_IMGPATH),
                        SPOT_ADDR = VALUES(SPOT_ADDR),
                        SPOT_INFOCENTER = VALUES(SPOT_INFOCENTER),
                        SPOT_PARKING = VALUES(SPOT_PARKING),
                        SPOT_USETIME = VALUES(SPOT_USETIME),
                        SPOT_TAGNAME = VALUES(SPOT_TAGNAME),
                        SPOT_DETAIL = VALUES(SPOT_DETAIL),
                        SPOT_LON = VALUES(SPOT_LON),
                        SPOT_LAT = VALUES(SPOT_LAT)
                """
                cursor.execute(query, (
                    row['id'],
                    row['city'],
                    row['cityCode'],
                    row['contentType'],
                    row['title'],
                    row['catchtitle'],
                    row['overView'],
                    row['treatMenu'],
                    row['conLike'],
                    row['conRead'],
                    row['conShare'],
                    row['imgPath'],
                    row['addr'],
                    row['info'],
                    row['parking'],
                    row['useTime'],
                    row['tagName'],
                    row['detail'],
                    row['lon'],
                    row['lat']
                ))

    except Exception as e:
        print(f"Error: {e}")
    finally:
        conn.commit()
        conn.close()